In [1]:
from dotenv import load_dotenv
import urllib.parse as urlparse
import requests
import json
import os


class UntappdApi:

    _clientId = ""
    _clientSecret = ""
    _baseUrl = "https://api.untappd.com/v4/"
    _keys = {}

    def __init__(self, clientId, clientSecret):
        self._clientId = clientId
        self._clientSecret = clientSecret
        self._keys = {
        "client_id": self._clientId,
        "client_secret": self._clientSecret
    }

    def findBrewery(self, brewery="", writeFile=False):
        methodUrl = self._baseUrl + "search/brewery"
        params = {
            "q": brewery
        }
        requestUrl = methodUrl + "?" + urlparse.urlencode({**params, **self._keys})
        response = requests.get(requestUrl)
        
        headers = response.headers
        counter = headers["X-Ratelimit-Remaining"]
        
        if response.status_code == 200:
            content = response.json()
            
            if content["response"]["found"] > 0:
                breweries = content["response"]["brewery"]["items"]
        
        return breweries, counter, response
    
    def breweryInfo(self, breweryId="", writeFile=False):
        methodUrl = self._baseUrl + "brewery/info/"
        requestUrl = methodUrl + str(breweryId) + "?" + urlparse.urlencode(self._keys)
        response = requests.get(requestUrl)
        
        headers = response.headers
        counter = headers["X-Ratelimit-Remaining"]
        
        if response.status_code == 200:
            content = response.json()
            
            brewery = content["response"]
        
        return brewery, counter, response


In [2]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="../.env")

untappd = UntappdApi(os.getenv("CLIENT_ID"),os.getenv("CLIENT_SECRET"))

breweries, counter, response = untappd.findBrewery("Community Beer Company")

breweries

[{'brewery': {'brewery_id': 48372,
   'beer_count': 213,
   'brewery_name': 'Community Beer Company',
   'brewery_slug': 'community-beer-company',
   'brewery_page_url': '/CommunityBeerCompanyTexas',
   'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-48372_9a99a.jpeg',
   'country_name': 'United States',
   'location': {'brewery_city': 'Dallas',
    'brewery_state': 'TX',
    'lat': 32.7909,
    'lng': -96.8158}}},
 {'brewery': {'brewery_id': 259927,
   'beer_count': 1,
   'brewery_name': 'The Community Beer Company',
   'brewery_slug': 'the-community-beer-company',
   'brewery_page_url': '/w/the-community-beer-company/259927',
   'brewery_label': 'https://untappd.akamaized.net/site/assets/images/temp/badge-brewery-default.png',
   'country_name': 'United States',
   'location': {'brewery_city': '', 'brewery_state': '', 'lat': 0, 'lng': 0}}}]

In [3]:
breweryId = breweries[0]["brewery"]["brewery_id"]

untappd = UntappdApi(os.getenv("CLIENT_ID"),os.getenv("CLIENT_SECRET"))
brewery, counter, response = untappd.breweryInfo(breweryId)

brewery

{'brewery': {'brewery_id': 48372,
  'brewery_name': 'Community Beer Company',
  'brewery_slug': 'community-beer-company',
  'brewery_page_url': '/CommunityBeerCompanyTexas',
  'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-48372_9a99a.jpeg',
  'brewery_label_hd': '',
  'country_name': 'United States',
  'brewery_in_production': 0,
  'is_independent': 1,
  'claimed_status': {'is_claimed': True,
   'claimed_slug': 'CommunityBeerCompanyTexas',
   'follow_status': False,
   'follower_count': 10003,
   'uid': 514817,
   'mute_status': 'none'},
  'beer_count': 213,
  'contact': {'twitter': 'CommunityBeerCo',
   'facebook': 'http://www.facebook.com/communitybeer',
   'instagram': 'communitybeerco',
   'url': 'http://www.communitybeer.com'},
  'brewery_type': 'Regional Brewery',
  'brewery_type_id': 11,
  'location': {'brewery_address': '1530 Inspiration Dr',
   'brewery_city': 'Dallas',
   'brewery_state': 'TX',
   'brewery_lat': 32.7909,
   'brewery_lng': -96.8158